In [1]:
import dask.dataframe as dd
import pandas as pd

In [9]:
data_dir = '../data/raw/'
df_sales = pd.read_csv(data_dir+'sales.csv', delimiter='\t')
df_cust = pd.read_csv(data_dir+'Customers.csv', delimiter='\t')
df_mail_train = pd.read_csv(data_dir+'Mailings1_2.csv')
df_demog = pd.read_csv(data_dir+'Demog.csv', delimiter='\t')
df_mail_test = pd.read_csv(data_dir+'Mailings3.txt',delimiter='\t')

In [15]:
print(df_sales.shape)
df_sales.head(3)

(3420829, 4)


,EVENTID,REFID,EVENT_DATE,AMOUNT
0,0,0,2006-12-21 12:00:00,17.907000
1,1,0,2006-12-25 12:00:00,17.400999
2,2,0,2007-01-26 12:00:00,13.277000


In [16]:
print(df_cust.shape)
df_cust.head(3)

(360000, 13)


,ID,SEX,MARITAL_STATUS,GEOID,EDUCATIONNUM,OCCUPATION,DATA1,DATA2,DATA3,NOM1,NOM2,NOM3,age
0,0,Male,Never-married,61,7,Machine-op-inspct,9.276,3.592,8.253,l,p,o,62
1,1,Male,Married-civ-spouse,70,3,Handlers-cleaners,5.836,4.574,5.507,u,r,x,78
2,2,Male,Never-married,53,7,Machine-op-inspct,8.998,4.184,8.593,n,q,o,69


In [17]:
print(df_mail_train.shape)
df_mail_train.head(3)

(60000, 4)


,REFID,KxIndex,REF_DATE,RESPONSE
0,0,1,2007-02-01 12:00:00,False
1,1,2,2007-02-01 12:00:00,False
2,2,3,2007-02-01 12:00:00,False


In [18]:
print(df_demog.shape)
df_demog.head(3)

(200, 21)


,GEOID,INHABITANTS_K,INCOME_K,A_VAR1,A_VAR2,A_VAR3,A_VAR4,A_VAR5,A_VAR6,A_VAR7,...,A_VAR9,A_VAR10,A_VAR11,A_VAR12,A_VAR13,A_VAR14,A_VAR15,A_VAR16,A_VAR17,A_VAR18
0,0,30.046,2631.468018,6.084,5.790,8.595,3.935,6.362,8.626,4.624,...,5.233,6.232,5.205,8.231,6.746,8.679,5.292,3.500,5.512,5.783
1,1,36.250,3012.745117,4.604,8.309,6.007,5.938,8.773,3.579,6.349,...,6.884,7.062,7.319,3.720,6.405,7.202,4.932,7.969,8.150,5.633
2,2,47.645,2192.410889,4.911,8.557,5.934,6.494,9.172,3.202,6.157,...,7.942,7.901,7.928,2.330,6.029,6.455,4.720,8.564,8.342,4.938


In [19]:
print(df_mail_test.shape)
df_mail_test.head(3)

(300000, 3)


,REFID,REF_DATE,RESPONSE
0,60000,2007-07-01 12:00:00,False
1,60001,2007-07-01 12:00:00,False
2,60002,2007-07-01 12:00:00,False


# Rossmann
https://www.kaggle.com/c/rossmann-store-sales/data



In [24]:
df_store = pd.read_csv('../data/raw/rossmann-store-sales/store.csv')
print(df_store.shape)
df_store.head(2)

(1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [25]:
df_train = pd.read_csv('../data/raw/rossmann-store-sales/train.csv')
print(df_train.shape)
df_train.head(2)

(1017209, 9)


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1


## Open ads

In [38]:
df = pd.read_csv('../data/raw/open_advertising/bids-dataset-2 2/US-market/traffic_estimates_20120524_0817483.csv',
                encoding='utf-16', delimiter='\t')




In [39]:
df.shape

(550, 16)

In [40]:
df.head(2)

,Campaign Name,Country,Geo Targeting,ID,Budget,Ad Group,Max. CPC,Keyword,Language,Keyword Type,Average Position,Average CPC,Clicks,CTR,Cost,Impressions
0,Draft campaign,NaN,NaN,NaN,10000000.0,NaN,NaN,NaN,NaN,NaN,1.0,2.34,282271.56,4.2%,660762.45,6714288.0
1,Draft campaign,NaN,United States,2840.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## amazon pricing

In [41]:
df = pd.read_csv('../data/raw/amazon_pricing/09-19-Amazon-Ranking-Analysis.csv')
df.shape

(6973, 14)

## Fb ads

In [42]:
df = pd.read_csv('../data/raw/fbpac-ads/fbpac-ads-en-US.csv')
df.shape


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(125281, 24)

In [43]:
df.head(2)


,id,html,political,not_political,title,message,thumbnail,created_at,updated_at,lang,...,suppressed,targets,advertiser,entities,page,lower_page,targetings,paid_for_by,targetedness,listbuilding_fundraising_proba
0,23843073208290387,"<div class=""_5pcr userContentWrapper""><div cla...",26,1,Richard Ojeda,"<p>Where I come from, leaders don't eat until ...",https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-10-19 03:06:32.454205+00,2018-10-25 17:32:38.994879+00,en-US,...,f,"[{""target"": ""Age"", ""segment"": ""27 and older""},...",Richard Ojeda,"[{""entity"": ""West Virginians"", ""entity_type"": ...",https://www.facebook.com/Ojedaforcongress/,https://www.facebook.com/ojedaforcongress/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",Committee to Elect Richard Ojeda,4.0,1.00000
1,23842985274260441,"<div class=""_5pcr userContentWrapper""><div cla...",0,0,For Our Future FL,<p>You should have received your Vote-by-Mail ...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-10-24 00:09:45.58062+00,2018-10-25 17:33:07.694581+00,en-US,...,f,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",For Our Future FL,"[{""entity"": ""ASAP"", ""entity_type"": ""Group""}]",https://www.facebook.com/ForOurFutureFL/,https://www.facebook.com/forourfuturefl/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",For Our Future (ForOurFuturePAC.org). Not auth...,4.0,0.22481


In [44]:
df.columns

Index(['id', 'html', 'political', 'not_political', 'title', 'message',
       'thumbnail', 'created_at', 'updated_at', 'lang', 'images',
       'impressions', 'political_probability', 'targeting', 'suppressed',
       'targets', 'advertiser', 'entities', 'page', 'lower_page', 'targetings',
       'paid_for_by', 'targetedness', 'listbuilding_fundraising_proba'],
      dtype='object')